In [1]:
import ipyparallel as ipp
import os
import numpy as np

In [2]:
rc = ipp.Client()

In [3]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [4]:
dview = rc[:]

In [5]:
dview.execute('import alfabeto_data.hmm_threading as ht')
dview.execute('import alfabeto_data.dissertation_images as di')
parallel_dict_ = {}
thing_number_ = 0
for cluster_ in range(2, 16):
    for mc_ in range(5):
        for mc_range_ in range(500):
            parallel_dict_[thing_number_] = {'cluster':cluster_, 'mc': mc_, 'mc_range': [mc_range_]}
            thing_number_ += 1

remaining_list = []
for num, data in parallel_dict_.items():
    for corpus in ['alfabeto_continuo', 'bach', 'palestrina']:
        for mode in ['major', 'minor']:
            if not os.path.exists('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/%s.pkl' % (corpus, mode, parallel_dict_[num]['cluster'],
                                                                     parallel_dict_[num]['mc'],
                                                                     parallel_dict_[num]['mc_range'][0])):
                if num not in remaining_list:
                    remaining_list.append(num)

print('remaining length:', len(remaining_list))

remaining length: 527


In [6]:
@dview.parallel(block=True)
def parallel_hmm(parallel_number):
    parallel_dict = {}
    thing_number = 0
    for cluster in range(2, 16):
        for mc in range(5):
            for mc_range in range(500):
                parallel_dict[thing_number] = {'cluster': cluster, 'mc': mc, 'mc_range': [mc_range]}
                thing_number += 1

    ht.hmm_pickle_generator('alfabeto continuo', di.GetAll.all_alf, di.modal_major, 'alfabeto_continuo',
                            'major', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')

    ht.hmm_pickle_generator('alfabeto continuo', di.GetAll.all_alf, di.modal_minor, 'alfabeto_continuo',
                            'minor', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')

    ht.hmm_pickle_generator('palestrina', di.palestrina_continuo, [(0, 9), (-1, 2)], 'palestrina',
                            'minor', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')
    ht.hmm_pickle_generator('bach', di.bach_continuo, di.tonal_minor, 'bach',
                            'minor', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')

    ht.hmm_pickle_generator('palestrina', di.palestrina_continuo, [(0, 0), (-1, 5)], 'palestrina',
                            'major', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')
    ht.hmm_pickle_generator('bach', di.bach_continuo, di.tonal_major, 'bach',
                            'major', parallel_dict[parallel_number]['mc'], parallel_dict[parallel_number]['cluster'],
                            parallel_dict[parallel_number]['mc_range'], '/home/daniel/Desktop/')

In [ ]:
print('imported stuff')
np.random.shuffle(remaining_list)
print(len(remaining_list))
parallel_hmm.map(remaining_list)

imported stuff
527


In [ ]:
print(len(set(remaining_list)))

In [ ]:
parallel_dict_[remaining_list[0]]

In [ ]:
from sklearn.externals import joblib

In [ ]:
x = joblib.load('/home/daniel/Desktop/hmm_mc/palestrina/minor/4/2MC.pkl')

In [ ]:
x